In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Load dataset

In [ ]:
# RUN THIS CELL TO IMPORT YOUR DATA. 

# Load dataset into a dataframe
### YOUR CODE HERE ###
df0 = pd.read_csv("/kaggle/input/hr-analytics-and-job-prediction/HR_comma_sep.csv")


# Display first few rows of the dataframe
### YOUR CODE HERE ###
df0.head()

### Data exploration (EDA and data cleaning)

In [ ]:
#Basic information about data
df0.info()

In [ ]:
#Basic descriptive statistics about data
df0.describe()

In [ ]:
df0.columns

Some of the columns have misspeling and for some reason few starts with a capital letter, for convinience we will rename those columns

In [ ]:
#Renaming the columns
df0 = df0.rename(columns={'time_spend_company':'years_spend_company',
                    'Work_accident':'work_accident',
                   'Department':'department',
                    'average_montly_hours': 'average_monthly_hours'})
#checking new names
df0.columns

### Check missing values

In [ ]:
df0.isna().sum()

There is no missing data

## Check duplicates

In [ ]:
df0.duplicated().sum()

Wow, thats a lot, should inspect further

In [ ]:
df0[df0.duplicated()].head(5)

There is very little chance for appearing 2 same rows on random when there is so many variable, so i think we shoud drop duplicates

In [ ]:
#drop duplicates and save clear data in a new variable
df = df0[~df0.duplicated()]
#check first few rows of new dataframe
df.head()


## Check outliers

In [ ]:
#it's easier to detect outliers by creating visualization

sns.boxplot(df['years_spend_company'], orient='h')

In [ ]:
# Determine the number of rows containing outliers

percentile25 = df['years_spend_company'].quantile(0.25)
percentile75 = df['years_spend_company'].quantile(0.75)

#interquantile range:
iqr = percentile75 - percentile25
lower_limit = percentile25 - 1.5*iqr
upper_limit = percentile75 + 1.5*iqr

print('Lower limit: ', lower_limit)
print('Upper limit: ', upper_limit)

outliers = df[(df['years_spend_company'] < lower_limit)|(df['years_spend_company'] > upper_limit)]
print('Number of outliers: ', len(outliers))

Certain types of models are more sensitive to outliers than others. On the model building stage i should consider that dataset has some outliers

## Data Exploration

In [ ]:
# numbers of people who left vs. stayed
left_v_stay = df['left'].value_counts()
print(left_v_stay)
#percentages of people who left vs. stayed
print('Stayed: ', round(left_v_stay[0]/len(df['left'])*100,3))
print('left: ', round(left_v_stay[1]/len(df['left'])*100,3))

Dataset is unbalanced, approximately 16.6% employees in this dataset left. But it is not extremely unbalanced and we can model without any class rebalancing

### Data visualizations

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,8))

sns.scatterplot(x=df['number_project'], y=df['satisfaction_level'], hue=df['left'], ax=ax[0])
ax[0].set_title('Satisfaction level vs number of projects')
sns.histplot(data=df, x='number_project', hue='left',multiple='dodge',shrink=3)
ax[1].set_title('Number of project histogram')
plt.show()

Interesting almost everyone who left has low satisfaction level and almost every one who had 7 project are left employees

People with 3 projects have the lowes number of leaving employees

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(data=df, x='average_monthly_hours', y='satisfaction_level', hue='left', alpha =0.5)
plt.axvline(x=160, color='r', label='160 hours/mo.', ls='--')
plt.legend(labels=['160 hours/mo.', 'left', 'stayed'])
plt.title('mothly hours by satisfaction level')

plt.show()

* data points are grouped in rectangles this could indicate that data is synthetic
* Also this plot shows us that a lot of left employees severly overworked and big group worked just a little bit less than average
* But most of left people have low satisfaction level

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (16,9))

sns.boxplot(data=df, x='years_spend_company', y='satisfaction_level', hue='left', ax=ax[0])
ax[0].set_title('Satisfaction by years spent in company')

sns.histplot(data=df, x='years_spend_company', hue='left', multiple='dodge', shrink=5, ax=ax[1])
ax[1].set_title('Years spent in company histogram')

plt.show()

* Employees who left fall into two general categories: dissatisfied employees with shorter years worked for company and very satisfied employees with medium-length years worked for company.
* Four-year employees who left seem to have an unusually low satisfaction level.
* The histogram shows that there are relatively few longer years_spend_company employees. It's possible that they're the higher-ranking, higher-paid employees.

In [ ]:
df.groupby(['left'])['satisfaction_level'].agg([np.mean,np.median])

As expected, the mean and median satisfaction scores of employees who left are lower than those of employees who stayed.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,9))
#long and short time working employees
long_yrs_comp = df[df['years_spend_company'] > 6]
short_yrs_comp = df[df['years_spend_company'] <=6]
#plot short time working employees and their salaries
sns.histplot(data=short_yrs_comp, x='years_spend_company', hue='salary', hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=3, ax=ax[0])
ax[0].set_title('salary of short-time working employees')

sns.histplot(data=long_yrs_comp, x='years_spend_company', hue='salary', hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=2, ax=ax[1])
ax[1].set_title('salary of long-time working employees')

plt.show()

* Very strange observation occured, there is no one who worked for 9 years in this company. 
* And as we can see employees who spent more years for this company proportionally tend to have higher salaries

In [ ]:
plt.figure(figsize=(16,3))

sns.scatterplot(data=df, x='average_monthly_hours', y='promotion_last_5years', hue='left', alpha=0.5)
plt.axvline(x=160, color='r', label='160 hrs/mo.', ls='--')
plt.legend(['160 hrs/mo.', 'left', 'stayed'])
plt.title('Monthly hours vs promotion')
plt.show()

* Very few people who were promoted left the company.
* And almost everyone who worked more than 280 hours and was not promoted left

In [ ]:
plt.figure(figsize=(16,9))

sns.heatmap(df.drop(columns=['department', 'salary']).corr(), vmin=-1, vmax=1, annot=True, cmap=sns.cubehelix_palette(as_cmap=True))

The correlation heatmap confirms that the number of projects, monthly hours, and evaluation scores all have some positive correlation with each other, and whether an employee leaves is negatively correlated with their satisfaction level.

Leaving is tied to longer working hours, many projects, and generally lower satisfaction levels. It can be ungratifying to work long hours and not receive promotions or good evaluation scores. There's a sizeable group of employees at this company who are probably burned out.

# Model Building

### Identify types of models
Since the variable we need to predict (whether an employees leaves the company) is categorical,
I choose to build 2 Tree-based machine learning models compare them and choose the best one.

## Modeling
Firstly i would like to make some new features that might help my model.

In [ ]:
#feature engeneering
df['hard_worker'] = np.where(df['average_monthly_hours'] > 300, 1, 0)

df['load'] = df['average_monthly_hours']*df['last_evaluation']

df['hours_per_project'] = df['average_monthly_hours']/df['number_project']
df.head(5)

We need to convert our categorical variablte to numeric, because model can only understand numeric ones

In [ ]:
df['salary'].replace({'low':1,
                       'medium':2,
                       'high':3},
                     inplace=True)
df.head(5)

In [ ]:
df=pd.get_dummies(df)
df.head()

### Data split
Now we are ready to build a model but first we need to split our data into train/validation/test sets(60,20,20)

In [ ]:
#Separate our X and y variables wich isolate features and target variables

y = df['left']

X = df.copy()
X = X.drop(columns=['left'])

#Split into train validation and test sets

X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.25, stratify=y_tr, random_state=42)

Verify the number of rows in each of the sets.

In [ ]:
print('Train:', X_train.shape, y_train.shape)
print('test:', X_test.shape, y_test.shape)
print('validation:', X_val.shape, y_val.shape)

## XGBClassifier
We begin with using GridSearchCV to tune the model

In [ ]:
xgb = XGBClassifier(objective='binary:logistic',random_state=42)

cv_params = {'n_estimators': [150,300,500],
            'max_depth': [3,4,5],
            'min_child_weight': [0.5,1,1.5],
            'learning_rate':[0.05,0.1,0.2],
            }

#dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1'}

xgb_cv = GridSearchCV(xgb, cv_params, scoring=scoring, cv=5, refit='recall')

Now we need to fit the model

In [ ]:
%%time
xgb_cv.fit(X_train,y_train)

Examine our best average score

In [ ]:
xgb_cv.best_score_

The best combination of hyperparameters

In [ ]:
xgb_cv.best_params_

Could be useful to try different hyperparameters to tune model but for now i would leave it like this

In [ ]:
def make_results(model_name:str, model_object, metric:str):
    '''
    model_name: is the name that you want to give to your model in the output table
    model_object: a fit GridSearchCV object
    metric: could be precision, recall, accuracy or f1
    
    returns pandas DataFrame with the precision, recall, accuracy and f1 scores 
    for the model with the best 'metric' score across all validation folds
    '''
    metric_dict = {'precision':'mean_test_precision',
                  'recall':'mean_test_recall',
                  'f1':'mean_test_f1',
                  'accuracy':'mean_test_accuracy'}
    # we need to get all the results from the CV
    cv_results = pd.DataFrame(model_object.cv_results_)
    
    #now we need to isolate row with the best mean metric score
    best_result = cv_results.loc[cv_results[metric_dict[metric]].idxmax(),:]
    
    precision = best_result.mean_test_precision
    recall = best_result.mean_test_recall
    f1 = best_result.mean_test_f1
    accuracy = best_result.mean_test_accuracy
    
    table = pd.DataFrame({'model_name':[model_name],
                         'precision':[precision],
                         'recall':[recall],
                         'f1':[f1],
                         'accuracy':[accuracy]},)
    return table

In [ ]:
results = make_results('XGBClassifier', xgb_cv, 'recall')
results

All of the scores are pretty high, i guess it would be helpfull to spend more time tunning the model to make recall score even higher

## Random Forest
We begin with using GridSearchCV to tune the model

In [ ]:
rf = RandomForestClassifier(random_state=42)

cv_params_rf = {'max_depth': [3,5,None],
             'max_features': ["sqrt"],
             'max_samples': [0.8, 0.9],
             'min_samples_leaf': [1,2,4],
             'min_samples_split': [3,4,6],
             'n_estimators': [50,75,100],
             }

scoring = {'accuracy', 'precision', 'recall', 'f1'}

rf_cv = GridSearchCV(rf, cv_params_rf, scoring=scoring, cv=5, refit='recall')

In [ ]:
%%time
rf_cv.fit(X_train,y_train)

In [ ]:
rf_cv.best_score_

In [ ]:
rf_cv.best_params_

In [ ]:
rf_res = make_results('RandomForest', rf_cv, 'recall')
results = pd.concat([results, rf_res])
results

Comparing to models we can see that XGBoost model performs a bit better on training data, now let's compare them on validation dataset

In [ ]:
xgb_y_pred = xgb_cv.best_estimator_.predict(X_val)

In [ ]:
def get_test_score(model_name:str, preds, y_test_data):
    
    
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    
    table = pd.DataFrame({'model_name':[model_name],
                         'precision':[precision],
                         'recall':[recall],
                         'f1':[f1],
                         'accuracy':[accuracy]},)
    return table

In [ ]:
xgb_val_scores = get_test_score('XGB_val', xgb_y_pred, y_val)

results = pd.concat([results, xgb_val_scores])

In [ ]:
rf_y_preds = rf_cv.best_estimator_.predict(X_val)

rf_val_scores = get_test_score('RF_val', rf_y_preds, y_val)
results = pd.concat([results, rf_val_scores])
results

XGB model made even better prediction on validation dataset
Now let's fit our best model with test data

In [ ]:
y_pred = xgb_cv.best_estimator_.predict(X_test)

xgb_final_score = get_test_score('XGB_best', y_pred, y_test)

results = pd.concat([results,xgb_final_score])
results

## Construct confusion matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)

disp = ConfusionMatrixDisplay(cm, display_labels=xgb_cv.classes_)
disp.plot()

Model predicts more False Negatives, that means that some employees may be identified as not at risk of leaving when in fact they will leave. But this is still a strong model

## Feature importance
we'll use plot_importance function to inspect the most important features of our final model

In [ ]:
plt.figure(figsize=(16,9))
plot_importance(xgb_cv.best_estimator_)

As we can see the most importans feature is satisfaction level as expected. But promotion in the last 5 years is not important feature but i thought it has high value on either employee leave or no. XGBoost model couldn't make more use of the department feature.

## Summary of model results


In [ ]:
print(results)

After conducting feature engineering and modeling two different models we have our champion model - XGB
that achived precision-score of 95.1%, recall-score of 93.2%, f1-score of 94,2% and accuracy-score of 98%

## Conclusion, Recommendations
The models and the feature importance extracted from the models confirm that employees at the company overwork a lot.

To retain employees company could change their working politics such as:
* Cap number of project that employee can work on
* Reward employees that overwork or create enviroment so that they couldn't overwork.
* Consider promoting employees who have been working for the company at least four years, or investigate why their satisfaction level is so low.